<a href="https://colab.research.google.com/github/Seif-Moharam/Music-Genre-Classification/blob/main/Music_Genre_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os

import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

from tqdm import tqdm

In [2]:
drive.mount("/content/drive")
path = "/content/drive/MyDrive/Colab Notebooks/Elevvo Internship/Music Genre Classification/Data/genres_original"

# This file seems corrupted and I wasn't able to work around
bad_file_path = "/content/drive/MyDrive/Colab Notebooks/Elevvo Internship/Music Genre Classification/Data/genres_original/jazz/jazz.00054.wav"

if os.path.exists(bad_file_path):
    os.remove(bad_file_path)
    print(f"Removed problematic file: {bad_file_path}")

Mounted at /content/drive


In [3]:
def extract_features(file_path):
  try:
    y, sr = librosa.load(file_path, duration=30)
  except Exception as e:
    print(f"Error loading {file_path}: {e}")
    return None

  features = {
      "zero_crossing_rate": np.mean(librosa.feature.zero_crossing_rate(y=y)),
      "rms_energy": np.mean(librosa.feature.rms(y=y)),
      "spectral_centroid_mean": np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),
      "spectral_centroid_std": np.std(librosa.feature.spectral_centroid(y=y, sr=sr)),
      "spectral_bandwidth_mean": np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)),
      "spectral_rolloff_mean": np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr)),
  }

  mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
  for i in range(20):
    features[f'mfcc_{i+1}_mean'] = np.mean(mfcc[i])
    features[f'mfcc_{i+1}_std'] = np.std(mfcc[i])

  chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
  features.update({
      'chroma_stft_mean': np.mean(chroma_stft),
      'chroma_stft_var': np.var(chroma_stft),
      'chroma_stft_std': np.std(chroma_stft),
  })

  y_harmonic, y_percussive = librosa.effects.hpss(y)
  features.update({
      'harmonic_mean': np.mean(y_harmonic),
      'percussive_mean': np.mean(y_percussive),
      'harmonic_ratio': np.mean(y_harmonic) / (np.mean(y_percussive) + 1e-6)
  })

  tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
  features['tempo'] = tempo

  return features

In [4]:
audio_files = []
for root, dirs, files in os.walk(path):
  for file in files:
    genre = os.path.basename(root)
    audio_files.append({
      'file_path': os.path.join(root, file),
      'genre': genre
      })

df_files = pd.DataFrame(audio_files)

In [5]:
features_list = []
for idx, row in tqdm(df_files.iterrows(), total=len(df_files)):
  features = extract_features(row['file_path'])
  features['genre'] = row['genre']
  features['filename'] = os.path.basename(row['file_path'])
  features_list.append(features)

df_features = pd.DataFrame(features_list)

100%|██████████| 999/999 [50:36<00:00,  3.04s/it]


In [7]:
# The previous cell took too long to run, so I made a CSV file to save time
df_features.to_csv('/content/drive/MyDrive/Colab Notebooks/Elevvo Internship/Music Genre Classification/Data/genres_original/features.csv', index=False)